# Heart Disease Exploration

Authors: Sophie Groenwold, Priyanka Banerjee

In [2]:
import numpy as np
import altair as alt
import pandas as pd

In [23]:
kaggle_df = pd.read_csv('heart.csv')
# kaggle_df.head(10)
kaggle_df = kaggle_df.sort_values(by=['age'])
# print(kaggle_df['trestbps'].shape)
# print(kaggle_df.shape)

kaggle_df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
72,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
58,34,1,3,118,182,0,0,174,0,0.0,2,0,2,1
125,34,0,1,118,210,0,1,192,0,0.7,2,0,2,1
239,35,1,0,126,282,0,0,156,1,0.0,2,0,3,0
65,35,0,0,138,183,0,1,182,0,1.4,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,71,0,2,110,265,1,0,130,0,0.0,2,1,2,1
151,71,0,0,112,149,0,1,125,0,1.6,1,0,2,1
129,74,0,1,120,269,0,0,121,1,0.2,2,1,2,1
144,76,0,2,140,197,0,2,116,0,1.1,1,0,2,1


In [24]:
cleveland_df = pd.read_csv('processed_cleveland.csv')
# cleveland_df.head(10)
cleveland_df = cleveland_df.sort_values(by=['a'])
cleveland_df

,a,b,c,d,e,f,g,h,i,j,k,l,m,n
132,29.0,1.0,2.0,130.0,204.0,0.0,2.0,202.0,0.0,0.0,1.0,0.0,3.0,0
101,34.0,1.0,1.0,118.0,182.0,0.0,2.0,174.0,0.0,0.0,1.0,0.0,3.0,0
225,34.0,0.0,2.0,118.0,210.0,0.0,0.0,192.0,0.0,0.7,1.0,0.0,3.0,0
283,35.0,1.0,2.0,122.0,192.0,0.0,0.0,174.0,0.0,0.0,1.0,0.0,3.0,0
117,35.0,0.0,4.0,138.0,183.0,0.0,0.0,182.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,71.0,0.0,2.0,160.0,302.0,0.0,0.0,162.0,0.0,0.4,1.0,2.0,3.0,0
103,71.0,0.0,3.0,110.0,265.0,1.0,2.0,130.0,0.0,0.0,1.0,1.0,3.0,0
233,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,0
257,76.0,0.0,3.0,140.0,197.0,0.0,1.0,116.0,0.0,1.1,2.0,0.0,3.0,0


In [20]:
# to view all the range of values for each feature

# for col in kaggle_df:
#     print(kaggle_df[col].unique())

# for col in cleveland_df:
#     print(cleveland_df[col].unique())

In [ ]:
# age = a
# sex = b
# trestbps = d (but weird -- cleveland has an extra row)
# chol = e
# fbs = f
# thalach = h
# oldpeak = j

# kaggle values missing their counterpart:
# - cp
# - restecg
# - exang
# - slope
# - ca
# - thal
# - target (which is what we would've replaced)

# cleveland values missing their counterpart:
# - c
# - g
# - i
# - k
# - l
# - m
# - n (which is what we would've used in place of target)

In [36]:
# I used this to compare the different columns from each dataframe
kaggle_df = kaggle_df.sort_values(by=['ca'])
cleveland_df = cleveland_df.sort_values(by=['l'])
# print(kaggle_df['restecg'].unique().shape)
# print(cleveland_df['j'].unique().shape)
print(kaggle_df['ca'].unique() == cleveland_df['l'].unique())

[False False False False False]
